<a href="https://colab.research.google.com/github/nsw2022/Android_BackUp/blob/master/%5B%EC%A1%B0%EB%85%81%EC%B9%B4%EC%84%B8%5D_ChromaDB_%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- ChromaDB Doc : https://docs.trychroma.com/

# 크로마를 사용한 기본 임베딩 검색 <br> (Basic embedding retrieval with Chroma)

이 노트북은 임베딩을 사용해 정보를 저장하고 검색하는 가장 기본적인 크로마 사용법을 보여드립니다. 이 핵심 빌딩 블록은 많은 강력한 AI 애플리케이션의 핵심입니다.
<br>
This notebook demonstrates the most basic use of Chroma to store and retrieve information using embeddings. This core building block is at the heart of many powerful AI applications.

## 임베딩이 뭔데? (What are embeddings?)

임베딩은 모든 종류의 데이터를 표현하는 인공지능 고유의 방식으로, 모든 종류의 인공지능 기반 도구 및 알고리즘을 사용하는 데 적합합니다. 임베딩은 텍스트, 이미지, 그리고 곧 오디오와 비디오를 나타낼 수 있습니다.

임베딩을 생성하려면 데이터를 임베딩 모델에 입력하면 숫자 벡터가 출력됩니다. 이 모델은 '유사한' 데이터(예: 비슷한 의미를 가진 텍스트 또는 비슷한 내용의 이미지)가 서로 다른 데이터보다 서로 더 가까운 벡터를 생성하도록 학습됩니다.

<br>

Embeddings are the A.I-native way to represent any kind of data, making them the perfect fit for working with all kinds of A.I-powered tools and algorithms. They can represent text, images, and soon audio and video.

To create an embedding, data is fed into an embedding model, which outputs vectors of numbers. The model is trained in such a way that 'similar' data, e.g. text with similar meanings, or images with similar content, will produce vectors which are nearer to one another, than those which are dissimilar.

## 임베딩과 검색기(Embeddings and retrieval)

임베딩의 유사성 속성을 사용하여 정보를 검색하고 검색할 수 있습니다. 예를 들어 특정 주제와 관련된 문서나 특정 이미지와 유사한 이미지를 찾을 수 있습니다. 키워드나 태그를 검색하는 대신 의미론적 의미가 유사한 데이터를 찾아 검색할 수 있습니다.

<br>
We can use the similarity property of embeddings to search for and retrieve information. For example, we can find documents relevant to a particular topic, or images similar to a given image. Rather than searching for keywords or tags, we can search by finding data with similar semantic meaning.


In [ ]:
%pip install -Uq chromadb numpy datasets

## 예시 데이터(Example Dataset)

데모로 허깅페이스에서 제공하는 SciQ 데이터 세트를 사용합니다.

데이터 세트 설명, HuggingFace 제공:

<br>

As a demonstration we use the [SciQ dataset](https://arxiv.org/abs/1707.06209), available from [HuggingFace](https://huggingface.co/datasets/sciq).

Dataset description, from HuggingFace:


> SciQ 데이터 세트에는 물리학, 화학, 생물학 등에 관한 13,679개의 크라우드소싱 과학 시험 문제가 포함되어 있습니다. 문제는 각각 4개의 답안 옵션이 있는 객관식 형식으로 되어 있습니다. 대부분의 문제에는 정답에 대한 근거를 제시하는 추가 단락이 제공됩니다.

<br>

> The SciQ dataset contains 13,679 crowdsourced science exam questions about Physics, Chemistry and Biology, among others. The questions are in multiple-choice format with 4 answer options each. For the majority of the questions, an additional paragraph with supporting evidence for the correct answer is provided.

이 노트북에서는 주어진 질문에 대한 근거 자료를 검색하는 방법을 보여드리겠습니다.

In this notebook, we will demonstrate how to retrieve supporting evidence for a given question.


In [ ]:
# Get the SciQ dataset from HuggingFace
from datasets import load_dataset

dataset = load_dataset("sciq", split="train")

print("Number of questions : ", len(dataset))

# Filter the dataset to only include questions with a support
dataset = dataset.filter(lambda x: x["support"] != "")

print("Number of questions with support: ", len(dataset))

Number of questions :  11679
Number of questions with support:  10481


## Chroma에 데이터 로드(Loading the data into Chroma)

Chroma에는 임베딩 모델이 내장되어 있어 텍스트를 간편하게 로드할 수 있습니다. 몇 줄의 코드만으로 SciQ 데이터 세트를 Chroma에 로드할 수 있습니다.

<br>

Chroma comes with a built-in embedding model, which makes it simple to load text.
We can load the SciQ dataset into Chroma with just a few lines of code.



In [ ]:
# Import Chroma and instantiate a client. The default Chroma client is ephemeral, meaning it will not save to disk.
# Chroma를 가져와서 클라이언트를 인스턴스화합니다. 기본 Chroma 클라이언트는 임시적이므로 디스크에 저장되지 않습니다.

import chromadb

client = chromadb.Client()

- 컬렉션은 임베딩, 문서 및 추가 메타데이터를 저장하는 곳입니다. 이름을 지정하여 컬렉션을 만들 수 있습니다:

In [ ]:
# Create a new Chroma collection to store the supporting evidence. We don't need to specify an embedding fuction, and the default will be used.
# 새 크로마 컬렉션을 만들어 증빙 자료를 저장합니다. 임베딩 함수를 지정할 필요는 없으며 기본값이 사용됩니다.
collection = client.create_collection("sciq_supports")

In [ ]:
dataset

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 10481
})

In [ ]:
dataset['support'][0]

'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'

- 크로마는 텍스트를 저장하고 토큰화, 임베딩, 인덱싱을 자동으로 처리합니다.

In [ ]:
# Embed and store the first 100 supports for this demo
# 이 데모에 대한 앞 100개의 support 임베딩 및 저장하기
collection.add(
    ids=[str(i) for i in range(0, 100)],  # IDs are just strings
    documents=dataset["support"][:100],
    metadatas=[{"type": "support"} for _ in range(0, 100)
    ],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 56.8MiB/s]


In [ ]:
collection

Collection(name=sciq_supports)

## 데이터 질의날리기(Querying the data)

데이터가 로드되면 Chroma를 사용하여 데이터 세트의 질문에 대한 근거를 찾을 수 있습니다. 이 예에서는 임베딩 유사성 점수에 따라 가장 관련성이 높은 결과를 검색합니다.

Chroma는 유사도 계산과 가장 관련성 높은 결과 찾기를 처리하므로 사용자는 애플리케이션 구축에 집중할 수 있습니다.

<br>

Once the data is loaded, we can use Chroma to find supporting evidence for the questions in the dataset.
In this example, we retrieve the most relevant result according to the embedding similarity score.

Chroma handles computing similarity and finding the most relevant results for you, so you can focus on building your application.


In [ ]:
results = collection.query(
    query_texts=dataset["question"][:10],
    n_results=1)

검색된 "supports"와 함께 쿼리 질문을 표시합니다.

we display the query questions along with their retrieved "supports"

In [ ]:
# Print the question and the corresponding support
for i, q in enumerate(dataset['question'][:10]):
    print(f"Question: {q}")
    print(f"Retrieved support: {results['documents'][i][0]}")
    print()

Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Retrieved support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.

Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Retrieved support: Without Coriolis Effect the global winds would blow north to south

## 다음엔 무엇을? (What's next?)

나만의 애플리케이션 구축을 시작하려면 크로마 [설명서](https://docs.trychroma.com/getting-started)를 확인하세요.

여기에 설명된 핵심 임베딩 기반 검색 기능은 문서와 [채팅하기](https://github.com/chroma-core/chroma/tree/main/examples/chat_with_your_documents) 위한 대규모 언어 모델과 [BabyAgi](https://github.com/yoheinakajima/babyagi) and [Voyager](https://github.com/MineDojo/Voyager). 와 같은 에이전트를 위한 메모리 등 많은 강력한 AI 애플리케이션의 핵심입니다.

Chroma는 이미  [LangChain](https://python.langchain.com/docs/integrations/vectorstores/chroma) and [LlamaIndex](https://gpt-index.readthedocs.io/en/latest/examples/vector_stores/ChromaIndexDemo.html)를 비롯한 많은 인기 있는 AI 애플리케이션 프레임워크와 통합되어 있습니다.

커뮤니티에 가입하여 자세히 알아보고 프로젝트에 대한 도움을 받으세요: [Discord](https://discord.gg/MMeYNTmh3x) | [Twitter](https://twitter.com/trychroma)

[채용 중](https://trychroma.notion.site/careers-chroma-9d017c3007c7478ebd85bad854101497?pvs=4)
입니다!

<br>


Check out the Chroma documentation to [get started](https://docs.trychroma.com/getting-started) with building your own applications.

The core embeddings based retrieval functionality demonstrated here is at the heart of many powerful AI applications, like using large language models with Chroma to [chat with your documents](https://github.com/chroma-core/chroma/tree/main/examples/chat_with_your_documents), as well as memory for agents like [BabyAgi](https://github.com/yoheinakajima/babyagi) and [Voyager](https://github.com/MineDojo/Voyager).

Chroma is already integrated with many popular AI applications frameworks, including [LangChain](https://python.langchain.com/docs/integrations/vectorstores/chroma) and [LlamaIndex](https://gpt-index.readthedocs.io/en/latest/examples/vector_stores/ChromaIndexDemo.html).

Join our community to learn more and get help with your projects: [Discord](https://discord.gg/MMeYNTmh3x) | [Twitter](https://twitter.com/trychroma)

We are [hiring](https://trychroma.notion.site/careers-chroma-9d017c3007c7478ebd85bad854101497?pvs=4)!